# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with
```bash
'mongoimport --db uk_food --collection establishments --drop --file establishments.json --jsonArray'

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:

# Check if the 'uk_food' database exists
if 'uk_food' in mongo.list_database_names():
    # Access the 'uk_food' database
    uk_food_db = mongo['uk_food']
    
    # Print the name of the database
    print(f"The '{uk_food_db.name}' database exists.")
else:
    print("The 'uk_food' database was not found.")

The 'uk_food' database exists.


In [11]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [13]:
# review the collections in our new database
collections = db.list_collection_names()

# Pretty print the list of collections
pprint(collections)

['establishments']


In [15]:
# review a document in the establishments collection
establishments_collection = db['establishments']

# Find one document in the 'establishments' collection
document = establishments_collection.find_one()

# Pretty print the document
pprint(document)

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6730125ebc16eb11b31d764d'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [17]:
# assign the collection to a variable
establishments = db['establishments']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [19]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "Name": "Penang Flavours",  # Name of the restaurant
    "BusinessType": "Restaurant",  # Type of business
    "LocalAuthorityName": "Greenwich",  # The local authority (location)
    "RatingValue": None,  # No rating yet (can be updated later)
    "Address": "123 Example Street, Greenwich",  # The restaurant's address (replace with real address)
    "Latitude": None,  # Latitude is not provided
    "Longitude": None,  # Longitude is not provided
    "HygieneScore": None,  # Hygiene score is not provided yet
    "BusinessTypeID": None  # BusinessTypeID can be filled later after querying the BusinessType collection
}

In [21]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

InsertOneResult(ObjectId('67303336e3bbd9ed9581d4c1'), acknowledged=True)

In [23]:
# Check that the new restaurant was inserted
new_document = establishments.find_one({"Name": "Penang Flavours"})
pprint(new_document)

{'Address': '123 Example Street, Greenwich',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': None,
 'HygieneScore': None,
 'Latitude': None,
 'LocalAuthorityName': 'Greenwich',
 'Longitude': None,
 'Name': 'Penang Flavours',
 'RatingValue': None,
 '_id': ObjectId('67303336e3bbd9ed9581d4c1')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [37]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},  # Filter for the correct BusinessType
    {"BusinessTypeID": 1, "BusinessType": 1}  # Project only BusinessTypeID and BusinessType
)

# Pretty print the result
pprint(business_type)


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6730125ebc16eb11b31d764d')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [39]:
# Update the new restaurant with the correct BusinessTypeID
penang_flavours_id = establishments.update_one(
    {"Name": "Penang Flavours"},
    {"$set": {"BusinessTypeID": 1}}
)

In [41]:
# Confirm that the new restaurant was updated
updated_document = establishments.find_one({"Name": "Penang Flavours"})
pprint(updated_document)

{'Address': '123 Example Street, Greenwich',
 'BusinessType': 'Restaurant',
 'BusinessTypeID': 1,
 'HygieneScore': None,
 'Latitude': None,
 'LocalAuthorityName': 'Greenwich',
 'Longitude': None,
 'Name': 'Penang Flavours',
 'RatingValue': None,
 '_id': ObjectId('67303336e3bbd9ed9581d4c1')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [43]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the result
print(f"Number of establishments in Dover: {dover_count}")


Number of establishments in Dover: 994


In [45]:
# Delete all documents where LocalAuthorityName is "Dover"
result = establishments.delete_many({"LocalAuthorityName": "Dover"})

# Print the number of deleted documents
print(f"Number of establishments deleted: {result.deleted_count}")


Number of establishments deleted: 994


In [47]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents({"LocalAuthorityName": "Dover"})

# Print the result
print(f"Number of establishments in Dover after removal: {dover_count_after}")

Number of establishments in Dover after removal: 0


In [49]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

# Print the document (or its fields)
pprint(remaining_document)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6730125ebc16eb11b31d7926'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [57]:
# Change the data type from String to Decimal for longitude and latitude
result = establishments.update_many(
    {"geocode.latitude": {"$type": "string"}, "geocode.longitude": {"$type": "string"}},  # Match documents where geocode.latitude and geocode.longitude are strings
    [
        {"$set": {
            "geocode.latitude": {"$toDecimal": "$geocode.latitude"},  # Convert geocode.latitude to Decimal128
            "geocode.longitude": {"$toDecimal": "$geocode.longitude"}  # Convert geocode.longitude to Decimal128
        }}
    ]
)

# Print the number of documents updated
print(f"Number of documents updated: {result.modified_count}")

Number of documents updated: 38785


In [59]:
# Verify by retrieving one document
updated_document = establishments.find_one()

# Check the data type and value of geocode.latitude and geocode.longitude
print(f"Updated geocode.latitude: {updated_document['geocode']['latitude']}, Type: {type(updated_document['geocode']['latitude'])}")
print(f"Updated geocode.longitude: {updated_document['geocode']['longitude']}, Type: {type(updated_document['geocode']['longitude'])}")


Updated geocode.latitude: 51.083812, Type: <class 'bson.decimal128.Decimal128'>
Updated geocode.longitude: 1.195625, Type: <class 'bson.decimal128.Decimal128'>


Use `update_many` to convert `RatingValue` to integer numbers.

In [61]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [63]:
# Change the data type from String to Integer for RatingValue
result = establishments.update_many(
    {"RatingValue": {"$type": "string"}},  # Match documents where RatingValue is a string
    [
        {"$set": {
            "RatingValue": {"$toInt": "$RatingValue"}  # Convert RatingValue to Integer
        }}
    ]
)

# Print the number of documents updated
print(f"Number of documents updated: {result.modified_count}")

Number of documents updated: 34694


In [65]:
# Check that the coordinates and rating value are now numbers
updated_document = establishments.find_one()

# Check the data type and value of RatingValue
print(f"Updated RatingValue: {updated_document['RatingValue']}, Type: {type(updated_document['RatingValue'])}")

Updated RatingValue: 5, Type: <class 'int'>
